# SSRS Expolratory Data Analysis

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
#plt.rcParams["figure.figsize"] = (width,height)
#sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from d2e2f.visualization.visualize import plot_map, plot_trips

In [ ]:
from functools import reduce
from operator import add

In [ ]:
datas = catalog.load("ssrs.data")
data = datas['SSRS15-01_322012079']()
data.index = pd.to_datetime(data.index)

In [ ]:
data['t'] = data.index - data.index[0]
data['latitude'].notnull()

## Cleaning

In [ ]:
def quantile_clean(data: pd.DataFrame, columns :list = None, alpha=0.001):
    
    if columns is None:
        columns = data.columns
    
    for key in columns:
        values = data[key]
        mask = ((values.quantile(alpha) < values) & 
        (values.quantile(1-alpha) > values)  )
        data = data.loc[mask]
        
    return data
        

In [ ]:
columns = data.columns[~data.isnull().all()]
data_clean = data[columns]

mask = ((-90 <= data['latitude']) & (data['latitude'] <=90) &
        (0 <= data['longitude']) & (data['longitude'] <=180)     
       )



data_clean = data_clean.loc[mask]
data_clean = quantile_clean(data_clean, columns=['latitude', 'longitude'])
data_clean = data_clean.fillna(method='backfill').fillna(method='ffill')

In [ ]:
renames = {key: reduce(add, key.split('_')[2:]) for key in data_clean.columns if len(key.split('_')[2:]) > 0}
data_clean.rename(columns=renames, inplace=True)
data_clean.head()

In [ ]:
data_clean.plot(x='longitude', y='latitude')

In [ ]:
data_clean['longitude'].quantile(0.001)

In [ ]:
data_clean.plot(y='SOGdshSA13')

In [ ]:
mask = ~((data_clean.columns.str.contains('COG') | data_clean.columns.str.contains('SOG')))
columns = list(data_clean.columns[mask])
columns.remove('latitude')
columns.remove('longitude')
data_plot = data_clean[columns].resample('1D').mean()
sns.pairplot(data_plot)

In [ ]:
data_plot.sum()

In [ ]:
sns.heatmap(data_plot.corr().abs(), cmap='gray_r')

In [ ]:
rpm_columns = list(data_clean.columns[data_clean.columns.str.contains('RPM')])
fuel_rate_columns = list(data_clean.columns[data_clean.columns.str.contains('FuelRate')])


In [ ]:
data_clean['rpm'] = data_clean[rpm_columns].mean(axis=1)
data_clean['fuel_rate'] = data_clean[fuel_rate_columns].mean(axis=1)
data_new = data_clean.drop(columns = rpm_columns + fuel_rate_columns)

renames = {
'SOGdshSA13' : 'sog',
'COGTruedshSA13' : 'cog',
}
data_new.rename(columns=renames, inplace=True)

In [ ]:
sns.pairplot(data_new.resample('100S').mean().drop(columns=['latitude','longitude','cog']), kind='scatter', plot_kws={'alpha':0.01})

#g = sns.PairGrid(data_new.resample('100S').mean().drop(columns=['latitude','longitude','cog']))
#g.map_upper(sns.scatterplot)
#g.map_lower(sns.scatterplot)

In [ ]:
mask = data_new['sog']>3
data_speed = data_new.loc[mask]

In [ ]:
sns.pairplot(data_speed.drop(columns=['latitude','longitude','cog']).resample('100S').mean(), kind='scatter', plot_kws={'alpha':0.1})


In [ ]:
plot_map(df=data_speed, zoom_start=9, color_key='sog')

In [ ]:
data_new['t'] = (data_new.index-data_new.index[0]).total_seconds()
    

In [ ]:
def insert_zero_at_time_jump(data_new):
    
    mask = data_new['t'].diff() > 30
    
    assert len(data_new.loc[mask]) / len(data_new) < 10**-2
    
    jumps = data_new.loc[mask]
    
    # Insering zeros before and after a jump to get correct integration.
    for index, jump in jumps.iterrows():
        i = data_new.index.get_loc(index)
        data_new.iloc[i][['sog','fuel_rate']] = 0
        data_new.iloc[i+1][['sog','fuel_rate']] = 0
        

In [ ]:
distance = scipy.integrate.trapz(data_new['sog'], dx=1.0)
distance

In [ ]:
consumption = scipy.integrate.trapz(data_new['fuel_rate']/3600, x=data_new['t'])
consumption

In [ ]:
consumption = scipy.integrate.trapz(data_new['fuel_rate']/3600, x=data_new['t'])
consumption

In [ ]:
consumption/distance*1852*1000

In [ ]:
consumption/data_new.iloc[-1]['t']*3600*1000

In [ ]:
data_new['sog'].mean()

In [ ]:
mask = data_new['sog']>0.25
data_speed = data_new.loc[mask]
distance = scipy.integrate.trapz(data_speed['sog'], dx=1.0)
distance

In [ ]:
consumption = scipy.integrate.trapz(data_speed['fuel_rate']/3600, dx=1.0)
consumption

In [ ]:
consumption/distance*1852*1000

In [ ]:
elapsed_time = len(data_speed)*1.0
consumption/elapsed_time*3600*1000

In [ ]:
data_speed['sog'].mean()

In [ ]:
import geopandas

In [ ]:
data = geopandas.GeoDataFrame(data_speed, 
                 geometry=geopandas.points_from_xy(data_speed.longitude, data_speed.latitude, crs="EPSG:4326"))
data = data.to_crs(epsg=3006)
data['distance'] = data.distance(data.shift())

In [ ]:
data['distance'].sum()

In [ ]:
data['distance'].sum()/distance

In [ ]:
data_speed['fuel_per_m'] = data_speed['fuel_rate']/data_speed['sog']

In [ ]:
from sklearn.svm import SVR
model = LinearRegression()
mask = data_speed['sog'] > 1
data = data_speed.loc[mask].resample('2S').mean().dropna()

def features(data):
    X = pd.DataFrame(index=data.index)
    X['sog'] = data['sog']
    X['sog**2'] = data['sog']**2
    X['sog**3'] = data['sog']**3
    X['sog**4'] = data['sog']**4
    X['sog**5'] = data['sog']**5
    X['sog**6'] = data['sog']**6
    X['sog**7'] = data['sog']**7
    
    return X

X = features(data)
#y = data['fuel_per_m']
y = data['fuel_rate']
model.fit(X,y)

In [ ]:
X_pred = pd.DataFrame()
X_pred['sog'] = np.linspace(data_speed['sog'].min(), data_speed['sog'].max(), 100)
X_pred = features(X_pred)

prediction = X_pred.copy()
prediction['fuel_rate'] = model.predict(X_pred)
prediction['fuel_per_m'] = prediction['fuel_rate']/X_pred['sog']



In [ ]:
fig,ax=plt.subplots()
data_speed.resample('10S').mean().plot(x='sog', y='fuel_per_m', style='.', alpha=0.05, ax=ax, label='__no_label__')
ax.set_ylim(data_speed['fuel_per_m'].quantile(0.01), 
            data_speed['fuel_per_m'].quantile(0.90))
ax.plot([],'b.', label='fuel rate per m')

prediction.plot(x='sog', y='fuel_per_m', ax=ax, label='regression')
ax.set_xticks(np.arange(0,int(np.ceil(data_speed['sog'].max())),1));
ax.grid(True)
ax.legend()

In [ ]:
data_speed['sog'].hist(bins=20)

In [ ]:
model = SVR(kernel='sigmoid', tol=0.0000000000001)
model.fit(X[['sog']], y)

In [ ]:
model.score(X[['sog']], y)

In [ ]:
model = LinearRegression()
model.fit(X, y)
model.score(X, y)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer
from sklearn.pipeline import make_pipeline

model = make_pipeline(SplineTransformer(n_knots=4, degree=4), Ridge(alpha=1e-3))
model.fit(X[['sog']],y)
model.score(X[['sog']],y)

In [ ]:
model=Ridge(alpha=0.5)
model.fit(X,y)
model.score(X,y)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=3, n_repeats=5, random_state=1)

standard_scaler = StandardScaler()
polynomial_features = PolynomialFeatures(degree=7)
select_k_best = SelectKBest(score_func=f_regression, k=7)
linear_regression = LinearRegression()

steps = [
    ('scaler', standard_scaler),
    ('polynomial_features', polynomial_features),
    ('select_k_best', select_k_best),
    ('linear_regression', linear_regression),
]

model = Pipeline(steps=steps)

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = dict()
grid['select_k_best__k'] = [i for i in range(10)]
grid['polynomial_features__degree'] = [i for i in range(3, 15)]


# define the grid search
search = GridSearchCV(estimator=model, param_grid=grid, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
# perform the search
search_result = search.fit(X[['sog']], y)


In [ ]:
steps = [
 ('transformer' ,SplineTransformer(n_knots=4, degree=3)),
 ('estimator', Ridge(alpha=1e-3)),
]
  
model = Pipeline(steps)

grid = dict()
grid['transformer__n_knots'] = [i for i in range(5)]
grid['transformer__degree'] = [i for i in range(10)]

# define the grid search
search = GridSearchCV(estimator=model, param_grid=grid, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
# perform the search
search_result = search.fit(X[['sog']], y)

In [ ]:
model = search_result.best_estimator_

In [ ]:
model.score(X[['sog']],y)

In [ ]:
model

In [ ]:
X_pred = pd.DataFrame()
X_pred['sog'] = np.linspace(data['sog'].min(), data['sog'].max(), 100)

prediction = X_pred.copy()
prediction['fuel_rate'] = model.predict(X_pred)
prediction['fuel_per_m'] = prediction['fuel_rate']/X_pred['sog']*1000/3600*1852

In [ ]:
fig,ax=plt.subplots()
data['fuel_per_m'] = data['fuel_rate']/data['sog']*1000/3600*1852
data.resample('10S').mean().plot(x='sog', y='fuel_per_m', style='.', alpha=0.05, ax=ax, label='__no_label__')

ax.set_ylim(data['fuel_per_m'].quantile(0.01), 
            data['fuel_per_m'].quantile(0.90))
ax.plot([],'b.', label='data')

prediction.plot(x='sog', y='fuel_per_m', ax=ax, label='regression')
ax.set_xticks(np.arange(0,int(np.ceil(data['sog'].max())),1));
ax.grid(True)
ax.legend()
ax.set_ylabel('Consumption per nautical mile [L/NM]')
ax.set_xlabel('Speed over ground [m/s]')